# Setting Source Folder

In [1]:
import os, sys

filedir = os.path.abspath(".")

currentdir = os.path.dirname(os.path.abspath("."))  # workaround to adding local modules

parentdir = os.path.dirname(os.path.abspath("" + currentdir))
print(f"CurrentDir: {currentdir}")
print(f"ParentDir: {parentdir}")

sys.path.insert(0, currentdir)


CurrentDir: /Users/renan.cristofalo/Documents/git/multimorbity_use
ParentDir: /Users/renan.cristofalo/Documents/git


# IMPORTAÇÃO DAS BIBLIOTECAS

In [2]:
# pip install tensorflow-metal==0.5.0 tensorflow-macos==2.9


In [3]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices("GPU")) > 0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

# For M1 computers = https://www.youtube.com/watch?v=5DgWvU0p2bk&ab_channel=JeffHeaton


Python Platform: macOS-14.1.2-arm64-arm-64bit
Tensor Flow Version: 2.9.0
Keras Version: 2.9.0

Python 3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:48:25) 
[Clang 14.0.6 ]
Pandas 1.3.0
Scikit-Learn 1.1.2
GPU is available


In [4]:
import pandas as pd
import numpy as np
import joblib
import os.path as osp
import logging

from src.visualization.data_visualization import *
from src.ml.model_selection import cross_validate_models, _create_metric_msg, create_df_results, RandomizedSearch, make_dnn_model
from src.ml.utils.preprocessors import select_preprocessor
from src.ml.utils.utils import write_joblib, read_joblib
from src.utils.utils import now_str
from src.ml.evaluation.evaluate import get_metric_result
from src.config import *

from sklearn.preprocessing import MinMaxScaler,StandardScaler, FunctionTransformer, PowerTransformer

from sklearn.pipeline import Pipeline
from sklearn.base import clone

from scikeras.wrappers import KerasClassifier

#Models
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, LGBMRegressor #conda install lightgbm==3.3.2
from catboost import CatBoostClassifier
# from ngboost import NGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier


from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

# import packages for imbalanced learn
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline as Imb_Pipeline
from imblearn.ensemble import RUSBoostClassifier, BalancedRandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline


In [5]:
N_FOLDS = 10
SEED = 123
MODELS_DIR = osp.join("models", "v3_segmentation")


# Definining Logs

In [6]:
def define_logs(logs_path):
    """Define the logs for the model selector

    Args:
        logs_path (str): Path to the log file
    """
    folder_path = osp.dirname(osp.abspath(logs_path))
    if not osp.exists(folder_path):
        os.makedirs(folder_path)

    log = logging.getLogger()
    log.setLevel(logging.INFO)
    fh = logging.FileHandler(filename=logs_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter(
        fmt="%(asctime)s %(levelname)s: %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
    )
    fh.setFormatter(formatter)
    log.addHandler(fh)

    return log


In [7]:
logger = define_logs(osp.join(MODELS_DIR, "logs", f"{now_str()}-model_selection.logs"))


In [8]:
import json


def is_jsonable(x):
    try:
        json.dumps(x)
        return True
    except (TypeError, OverflowError):
        return False


# Model Training

## Defining Strategies

In [17]:
bins_idade_10_years = [-float("inf")] + np.arange(20, 81, 10).tolist() + [float("inf")]
labels_idade_10_years = (
    ["<20"]
    + [
        f"{bins_idade_10_years[i]:.0f}_{bins_idade_10_years[i+1]-1:.0f}"
        for i in range(len(bins_idade_10_years) - 1)
    ][1:-1]
    + [">80"]
)


bins_grupos_etarios = [-float("inf"), float("inf")]

segments = {
    # "idade_10_years": {
    #     "column": "idade",
    #     "bins": bins_idade_10_years,
    #     "labels": labels_idade_10_years,
    # },
    # "idade_groups": {
    #     "column": "idade",
    #     "bins": [-float("inf"), 40, 60, float("inf")],
    #     "labels": ["<40", "40_59", ">60"],
    # },
    # "idade_quantiles": {
    #     "column": "idade",
    #     "qbins": 4,
    #     "labels": ["q1", "q2", "q3", "q4"],
    # },
    # "plano": {
    #     "column": "plano",
    # },
    # "cor2": {
    #     "column": "cor2",
    # },
    # "multi": {
    #     "column": "multi_calc",
    #     "bins": np.arange(0, 8, 1).tolist() + [float("inf")],
    #     "labels": [str(i) for i in np.arange(0, 8, 1).tolist()][:-1] + [">6"],
    # },
    "multi2": {
        "column": "multi_calc",
        "bins": [0, 1, 2] + [float("inf")],
        "labels": ["0", "1"] + [">=2"],
    },
}


In [18]:
FS_MUST2_PATH = osp.join("..", "data", "feature_selection", "fs_must2")

STRATEGIES = {
    "FS_MUST2": {
        "description": "Training with filtered features selected by the Feature Selector",
        "df_train": osp.join(FS_MUST2_PATH, "df_train.parquet"),
        "df_test": osp.join(FS_MUST2_PATH, "df_test.parquet"),
    },
}
for strategy, dict_strategy in STRATEGIES.items():
    logger.info(f"\n*************** {strategy} ***************\n")
    logger.info(
        json.dumps(
            {
                k: (str(v) if not is_jsonable(v) else v)
                for k, v in dict_strategy.items()
            },
            sort_keys=False,
            indent=3,
            skipkeys=True,
        )
    )

    # getting strategy parameters
    df_train_path = dict_strategy.get("df_train", None)
    df_test_path = dict_strategy.get("df_test", None)

    # reading files
    df_train = pd.read_parquet(df_train_path)
    df_test = pd.read_parquet(df_test_path)

    logger.info(
        f"\ndf_train shape: {df_train.shape} \n df_test shape: {df_test.shape}\n"
    )


In [19]:
df_train.describe()


,ansi_cut,asmbrq,nserv,rendasetor,idade,depre_cut,multi_calc,tvs,sent,altura,bmi,popw,cancer,pronto_atendimento
count,"2,209.00","2,211.00","2,211.00","2,210.00","2,211.00","2,209.00","2,211.00","2,166.00","2,167.00","2,211.00","2,211.00","2,211.00","2,210.00","2,211.00"
mean,0.17,0.09,0.68,"1,821.34",50.97,0.19,1.67,153.86,246.79,164.27,28.00,42.71,0.04,0.49
std,0.38,0.29,3.37,"1,106.83",16.37,0.39,1.76,134.94,182.25,9.19,5.35,18.53,0.20,0.50
min,0.00,0.00,0.00,631.00,18.00,0.00,0.00,0.00,0.00,130.00,14.57,25.57,0.00,0.00
25%,0.00,0.00,0.00,"1,252.00",38.00,0.00,0.00,60.00,120.00,158.00,24.24,29.95,0.00,0.00
50%,0.00,0.00,0.00,"1,512.00",52.00,0.00,1.00,120.00,210.00,164.00,27.53,35.58,0.00,0.00
75%,0.00,0.00,1.00,"1,916.00",63.00,0.00,3.00,180.00,305.00,170.00,30.86,49.88,0.00,1.00
max,1.00,1.00,99.00,"9,150.00",96.00,1.00,13.00,"1,440.00","1,200.00",197.00,50.81,94.82,1.00,1.00


## Defining Metrics

In [20]:
metrics = {
    "accuracy": "accuracy",
    "balanced_acc": "balanced_accuracy",
    "F1": "f1",
    "Precision": "precision",
    "recall": "recall",
    "roc_auc": "roc_auc",
}


## Defining Sklearn Models

In [21]:
models = []

# Make a pipeline for the model, scaling and making a cross validation for each of them
models.append({"name": "Dummy", "model": DummyClassifier()})

models.append({"name": "LogisticRegression", "model": LogisticRegression()})

models.append(
    {
        "name": "DecisionTreeClassifier",
        "model": DecisionTreeClassifier(
            min_samples_split=0.005, min_samples_leaf=0.001, random_state=SEED
        ),
    }
)

models.append(
    {
        "name": "RandomForestClassifier",
        "model": RandomForestClassifier(
            max_depth=7,
            random_state=SEED,
            min_samples_split=0.01,
            min_samples_leaf=0.005,
        ),
    }
)

models.append(
    {
        "name": "ExtraTreesClassifier",
        "model": ExtraTreesClassifier(
            max_depth=7,
            random_state=SEED,
            min_samples_split=0.01,
            min_samples_leaf=0.005,
        ),
    }
)

models.append(
    {
        "name": "GradientBoostingClassifier",
        "model": GradientBoostingClassifier(
            max_depth=7,
            random_state=SEED,
            min_samples_split=0.01,
            min_samples_leaf=0.005,
        ),
    }
)

models.append(
    {
        "name": "XGBoostClassifier",
        "model": XGBClassifier(max_depth=7, random_state=SEED),
    }
)

models.append(
    {"name": "XGBoostClassifier_Default", "model": XGBClassifier(random_state=SEED)}
)

models.append(
    {
        "name": "LGBMClassifier",
        "model": LGBMClassifier(max_depth=7, verbose=0, random_state=SEED),
    }
)

models.append(
    {"name": "LGBMClassifier_Default", "model": LGBMClassifier(random_state=SEED)}
)

models.append(
    {
        "name": "CatBoostClassifier",
        "model": CatBoostClassifier(max_depth=7, verbose=0, random_state=SEED),
    }
)

models.append(
    {
        "name": "CatBoostClassifier_Default",
        "model": CatBoostClassifier(random_state=SEED),
    }
)

models.append(
    {
        "name": "BalancedRandomForestClassifier",
        "model": BalancedRandomForestClassifier(random_state=SEED),
    }
)

models.append(
    {"name": "RUSBoostClassifier", "model": RUSBoostClassifier(random_state=SEED)}
)

models.append(
    {
        "name": "VotingClassifier",
        "model": VotingClassifier(
            estimators=[
                ("rf", RandomForestClassifier(n_estimators=10, random_state=42)),
                ("lgbm", LGBMClassifier(max_depth=10)),
                ("catboost", CatBoostClassifier(max_depth=10, verbose=False)),
                ("xgboost", XGBClassifier(max_depth=10, random_state=SEED)),
            ],
            voting="soft",
        ),
    }
)

models.append(
    {
        "name": "StackingClassifier",
        "model": StackingClassifier(
            estimators=[
                ("rf", RandomForestClassifier(n_estimators=10, random_state=42)),
                ("xgboost", XGBClassifier(max_depth=10, random_state=SEED)),
                ("lgbm", LGBMClassifier(max_depth=10)),
            ]
        ),
    }
)


## Defining Tensorflow Models

In [22]:
# models = []

PATIENCE = 5
NUM_EPOCHS = 30

BATCH_SIZE = 1024
seed = 42
verbose = 0


callbacks = {
    "EarlyStopping": tf.keras.callbacks.EarlyStopping,
    # "ReduceLROnPlateau": tf.keras.callbacks.ReduceLROnPlateau, # can't pickle
}
tf_metrics = ["accuracy"]  # https://www.tensorflow.org/api_docs/python/tf/keras/metrics

models.append(
    {
        "name": "DNN_01",
        "model": KerasClassifier(
            model=make_dnn_model,
            optimizer="adam",
            network_layers=(1024, 1024, 512, 512),
            dropout_rate=0.3,
            epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=callbacks,
            callbacks__EarlyStopping__monitor="val_loss",
            callbacks__EarlyStopping__patience=PATIENCE,
            callbacks__EarlyStopping__restore_best_weights=True,
            validation_split=1 / N_FOLDS,
            verbose=0,
        ),
    }
)

models.append(
    {
        "name": "DNN_02",
        "model": KerasClassifier(
            model=make_dnn_model,
            optimizer="adam",
            network_layers=(1024, 512, 256, 128, 64, 32),
            dropout_rate=1 / N_FOLDS,
            epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=callbacks,
            callbacks__EarlyStopping__monitor="val_loss",
            callbacks__EarlyStopping__patience=PATIENCE,
            callbacks__EarlyStopping__restore_best_weights=True,
            validation_split=1 / N_FOLDS,
            verbose=0,
        ),
    }
)

models.append(
    {
        "name": "DNN_03",
        "model": KerasClassifier(
            model=make_dnn_model,
            optimizer="adam",
            network_layers=(128, 64, 32),
            dropout_rate=0.1,
            epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=callbacks,
            callbacks__EarlyStopping__monitor="val_loss",
            callbacks__EarlyStopping__patience=PATIENCE,
            callbacks__EarlyStopping__restore_best_weights=True,
            validation_split=1 / N_FOLDS,
            verbose=0,
        ),
    }
)

models.append(
    {
        "name": "DNN_04",
        "model": KerasClassifier(
            model=make_dnn_model,
            optimizer="adam",
            network_layers=(32, 16, 8),
            dropout_rate=0.1,
            epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=callbacks,
            callbacks__EarlyStopping__monitor="val_loss",
            callbacks__EarlyStopping__patience=PATIENCE,
            callbacks__EarlyStopping__restore_best_weights=True,
            validation_split=1 / N_FOLDS,
            verbose=0,
        ),
    }
)


models.append(
    {
        "name": "VotingClassifierDNN_01",
        "model": VotingClassifier(
            estimators=[
                ("rf", RandomForestClassifier(n_estimators=10, random_state=42)),
                ("lgbm", LGBMClassifier(max_depth=10)),
                (
                    "DNN_01",
                    KerasClassifier(
                        model=make_dnn_model,
                        optimizer="adam",
                        network_layers=(1024, 1024, 512, 512),
                        dropout_rate=0.3,
                        epochs=NUM_EPOCHS,
                        batch_size=BATCH_SIZE,
                        callbacks=callbacks,
                        callbacks__EarlyStopping__monitor="val_loss",
                        callbacks__EarlyStopping__patience=PATIENCE,
                        callbacks__EarlyStopping__restore_best_weights=True,
                        validation_split=1 / N_FOLDS,
                        verbose=0,
                    ),
                ),
            ],
            voting="soft",
        ),
    }
)


## Running Models' Training

In [23]:
for strategy, dict_strategy in STRATEGIES.items():
    # logging strategy
    logger.info(f"*************** {strategy} ***************")
    logger.info(
        json.dumps(
            {
                k: (str(v) if not is_jsonable(v) else v)
                for k, v in dict_strategy.items()
            },
            sort_keys=False,
            indent=4,
            skipkeys=True,
        )
    )

    # getting strategy parameters
    df_train_path = dict_strategy.get("df_train", None)
    df_test_path = dict_strategy.get("df_test", None)
    imbalanced_sampler = dict_strategy.get("imbalanced_sampler", None)

    for segment, segment_dict in segments.items():
        logger.info(f"*************** {segment} ***************")

        col = segment_dict.get("column", None)
        bins = segment_dict.get("bins", None)
        qbins = segment_dict.get("qbins", None)
        labels = segment_dict.get("labels", None)

        # reading files
        df_train_raw = pd.read_parquet(df_train_path)
        df_test_raw = pd.read_parquet(df_test_path)

        if col == "cor2" and "cor2" not in df_train_raw.columns:
            df_train_raw["cor2"] = df_train_raw["cor"].apply(
                lambda x: "branca" if x.lower() == "branca" else "outra"
            )
            df_test_raw["cor2"] = df_test_raw["cor"].apply(
                lambda x: "branca" if x.lower() == "branca" else "outra"
            )
            df_train_raw = df_train_raw.drop(["cor"], axis=1)
            df_test_raw = df_test_raw.drop(["cor"], axis=1)

        # Bin the age groups in 5-year intervals
        if bins:
            df_train_raw[col + "_group"] = pd.cut(
                df_train_raw[col], bins=bins, labels=labels, right=False
            )
            df_test_raw[col + "_group"] = pd.cut(
                df_test_raw[col], bins=bins, labels=labels, right=False
            )
        elif qbins:
            df_train_raw[col + "_group"] = pd.qcut(
                df_train_raw[col], q=qbins, labels=labels
            )
            df_test_raw[col + "_group"] = pd.qcut(
                df_test_raw[col], q=qbins, labels=labels
            )
        else:
            df_train_raw[col + "_group"] = df_train_raw[col]
            df_test_raw[col + "_group"] = df_test_raw[col]
            labels = df_train_raw[col + "_group"].unique().tolist()

        for label_ in labels:
            df_train = df_train_raw[df_train_raw[col + "_group"] == label_]
            df_test = df_test_raw[df_test_raw[col + "_group"] == label_]

            X_train_ = df_train.drop([TARGET, col, col + "_group"], axis=1)
            y_train_ = df_train[TARGET]
            X_test_ = df_test.drop([TARGET, col, col + "_group"], axis=1)
            y_test_ = df_test[TARGET]

            logger.info(f"df_train shape: {df_train.shape}")
            logger.info(f"df_test shape: {df_test.shape}")

            # Adding preprocessor
            preprocessor = None
            preprocessor = select_preprocessor("TargetEncoding_all", X=X_train_)

            strategy_models = []
            if imbalanced_sampler:
                df_train = pd.read_parquet(df_train_path)
                X_train_ = df_train.drop([TARGET], axis=1)
                y_train_ = df_train[TARGET]

                for model in models:
                    strategy_models.append(
                        {
                            "name": strategy + "_" + model["name"],
                            "model": Imb_Pipeline(
                                steps=[
                                    ("preprocessor", preprocessor),
                                    ("sampling", imbalanced_sampler),
                                    ("model", model["model"]),
                                ]
                            ),
                        }
                    )
            else:
                for model in models:
                    strategy_models.append(
                        {
                            "name": strategy + "_" + model["name"],
                            "model": Pipeline(
                                steps=[
                                    ("preprocessor", preprocessor),
                                    ("model", model["model"]),
                                ]
                            ),
                        }
                    )

            names, results, df_results = cross_validate_models(
                models=strategy_models,
                X_train=X_train_,
                y_train=y_train_,
                X_test=X_test_,
                y_test=y_test_,
                metrics=metrics,
                n_folds=N_FOLDS,
                models_dir=osp.join(MODELS_DIR, segment, label_),
            )


  0%|          | 0/21 [00:00<?, ?it/s]


AVERAGES - FS_MUST2_DUMMY:
 - ACCURACY               ->  - TRAIN: 0.59 (+/- 0.00) | VALIDATION: 0.59 (+/- 0.01) | TEST: 0.59 (+/- 0.00)
 - BALANCED_ACC           ->  - TRAIN: 0.50 (+/- 0.00) | VALIDATION: 0.50 (+/- 0.00) | TEST: 0.50 (+/- 0.00)
 - F1                     ->  - TRAIN: 0.00 (+/- 0.00) | VALIDATION: 0.00 (+/- 0.00) | TEST: 0.00 (+/- 0.00)
 - PRECISION              ->  - TRAIN: 0.00 (+/- 0.00) | VALIDATION: 0.00 (+/- 0.00) | TEST: 0.00 (+/- 0.00)
 - RECALL                 ->  - TRAIN: 0.00 (+/- 0.00) | VALIDATION: 0.00 (+/- 0.00) | TEST: 0.00 (+/- 0.00)
 - ROC_AUC                ->  - TRAIN: 0.50 (+/- 0.00) | VALIDATION: 0.50 (+/- 0.00) | TEST: 0.50 (+/- 0.00)
 - FIT TIME FOR EACH FOLD ->  - TRAIN: 0.17s (+/- 0.03s)



  5%|▍         | 1/21 [00:04<01:35,  4.75s/it]


AVERAGES - FS_MUST2_LOGISTICREGRESSION:
 - ACCURACY               ->  - TRAIN: 0.78 (+/- 0.01) | VALIDATION: 0.57 (+/- 0.06) | TEST: 0.59 (+/- 0.02)
 - BALANCED_ACC           ->  - TRAIN: 0.75 (+/- 0.01) | VALIDATION: 0.54 (+/- 0.06) | TEST: 0.56 (+/- 0.02)
 - F1                     ->  - TRAIN: 0.70 (+/- 0.01) | VALIDATION: 0.41 (+/- 0.08) | TEST: 0.44 (+/- 0.03)
 - PRECISION              ->  - TRAIN: 0.77 (+/- 0.01) | VALIDATION: 0.46 (+/- 0.09) | TEST: 0.50 (+/- 0.03)
 - RECALL                 ->  - TRAIN: 0.64 (+/- 0.01) | VALIDATION: 0.37 (+/- 0.08) | TEST: 0.39 (+/- 0.03)
 - ROC_AUC                ->  - TRAIN: 0.86 (+/- 0.00) | VALIDATION: 0.58 (+/- 0.06) | TEST: 0.58 (+/- 0.01)
 - FIT TIME FOR EACH FOLD ->  - TRAIN: 0.18s (+/- 0.00s)



 10%|▉         | 2/21 [00:09<01:32,  4.86s/it]

...

2023-12-04 11:28:35.736015: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x763x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x763x1x1xi1>'
l


AVERAGES - FS_MUST2_VOTINGCLASSIFIERDNN_01:
 - ACCURACY               ->  - TRAIN: 1.00 (+/- 0.00) | VALIDATION: 0.58 (+/- 0.06) | TEST: 0.56 (+/- 0.02)
 - BALANCED_ACC           ->  - TRAIN: 1.00 (+/- 0.00) | VALIDATION: 0.57 (+/- 0.06) | TEST: 0.55 (+/- 0.02)
 - F1                     ->  - TRAIN: 1.00 (+/- 0.00) | VALIDATION: 0.65 (+/- 0.05) | TEST: 0.63 (+/- 0.02)
 - PRECISION              ->  - TRAIN: 1.00 (+/- 0.00) | VALIDATION: 0.61 (+/- 0.04) | TEST: 0.60 (+/- 0.01)
 - RECALL                 ->  - TRAIN: 1.00 (+/- 0.00) | VALIDATION: 0.70 (+/- 0.06) | TEST: 0.67 (+/- 0.03)
 - ROC_AUC                ->  - TRAIN: 1.00 (+/- 0.00) | VALIDATION: 0.60 (+/- 0.07) | TEST: 0.58 (+/- 0.01)
 - FIT TIME FOR EACH FOLD ->  - TRAIN: 4.20s (+/- 0.11s)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


2023-12-04 11:28:46.249725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x847x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x847x1x1xi1>'
l

INFO:tensorflow:Assets written to: ram:///var/folders/76/wty8h2t54v58m7s7y_zrmlt00000gn/T/tmp2nve85fd/assets
INFO:tensorflow:Assets written to: ram:///var/folders/76/wty8h2t54v58m7s7y_zrmlt00000gn/T/tmp_sfey9vv/assets
INFO:tensorflow:Assets written to: ram:///var/folders/76/wty8h2t54v58m7s7y_zrmlt00000gn/T/tmpoebr5dk2/assets
INFO:tensorflow:Assets written to: ram:///var/folders/76/wty8h2t54v58m7s7y_zrmlt00000gn/T/tmpg7e_xx58/assets
INFO:tensorflow:Assets written to: ram:///var/folders/76/wty8h2t54v58m7s7y_zrmlt00000gn/T/tmpucnetd0c/assets
INFO:tensorflow:Assets written to: ram:///var/folders/76/wty8h2t54v58m7s7y_zrmlt00000gn/T/tmp38eyu30e/assets
INFO:tensorflow:Assets written to: ram:///var/folders/76/wty8h2t54v58m7s7y_zrmlt00000gn/T/tmp45v9gi2u/assets
INFO:tensorflow:Assets written to: ram:///var/folders/76/wty8h2t54v58m7s7y_zrmlt00000gn/T/tmp_9yrinml/assets
INFO:tensorflow:Assets written to: ram:///var/folders/76/wty8h2t54v58m7s7y_zrmlt00000gn/T/tmp5ypmhwdw/assets
INFO:tensorflow:Ass

100%|██████████| 21/21 [09:21<00:00, 26.74s/it]


In [ ]:
trained_models, results = {}, {}
for strategy, dict_strategy in STRATEGIES.items():
    for model in models:
        try:
            trained_models[strategy + "_" + model["name"]] = read_joblib(
                osp.join(MODELS_DIR, strategy + "_" + model["name"], "model.p")
            )
            metric_dict = read_joblib(
                osp.join(MODELS_DIR, strategy + "_" + model["name"], "metrics.p")
            )
            results.update(metric_dict)
        except:
            pass

    write_joblib(results, osp.join(MODELS_DIR, f"results.p"))
    results = read_joblib(osp.join(MODELS_DIR, "results.p"))


In [ ]:
for name, model_metric in results.items():
    msg_metrics = _create_metric_msg(model_metric)

    # print of the result of each model
    print(f"\nAVERAGES - {name.upper()}:\n" + msg_metrics)
    logger.info(f"\nAVERAGES - {name.upper()}:\n" + msg_metrics)


In [ ]:
df_results = create_df_results(results, models=None, slim=False)
df_results_slim = create_df_results(results, models=None, slim=True)


In [ ]:
metrics = df_results.select_dtypes(float).columns.to_list()

compare_model_results(
    df=df_results,
    names="models",
    metrics=metrics,
    save_path=osp.join("figures", "compare_models.png"),
)


In [ ]:
metrics = df_results_slim.select_dtypes(float).columns.to_list()

compare_model_results(
    df=df_results_slim,
    names="models",
    metrics=metrics,
    save_path=osp.join("figures", "compare_models_slim.png"),
    hue="set",
)


# Analyzing the Results

In [ ]:
df_results.sample(10)


In [ ]:
df_results.groupby(["models"]).mean().nlargest(10, "VALIDATION_ROC_AUC").sort_values(
    "VALIDATION_ROC_AUC", ascending=False
)


In [ ]:
df_results.groupby(["models"]).mean().nlargest(5, "VALIDATION_F1").sort_values(
    "VALIDATION_F1", ascending=False
)


In [ ]:
df_results.groupby(["models"]).mean().nlargest(
    5, "VALIDATION_BALANCED_ACC"
).sort_values("VALIDATION_BALANCED_ACC", ascending=False)


In [ ]:
SELECTED_MODELS = sorted(
    [
        col
        for col in results.keys()
        if "Dummy" not in col and results[col]["ROC_AUC"]["VALIDATION"].mean() > 0.65
    ]
)
THRESHOLD = 0.5


In [ ]:
# reading files

df_train = pd.read_parquet(osp.join(ALL_PATH, "df_train.parquet"))
df_test = pd.read_parquet(osp.join(ALL_PATH, "df_test.parquet"))

logger.info(f"df_train shape: {df_train.shape}")
logger.info(f"df_test shape: {df_test.shape}")

X_train = df_train.drop([TARGET], axis=1)
y_train = df_train[TARGET]
X_test = df_test.drop([TARGET], axis=1)
y_test = df_test[TARGET]


In [ ]:
predictions = {}
predictions_proba = {}
for name, model in trained_models.items():
    if name in SELECTED_MODELS:
        predictions[name] = model.predict(X_test)
        predictions_proba[name] = model.predict_proba(X_test)[:, 1]


In [ ]:
import matplotlib.pyplot as plt
import math

ncols = 4

size = len(predictions)
rows = math.ceil(size / ncols)
width, height = (10, 6)

fig = plt.figure(figsize=(min(ncols * width, 50), rows * height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

for idx, (name, y_pred) in enumerate(predictions.items(), start=1):
    ax = fig.add_subplot(rows, ncols, idx)
    plot_confusion_matrix(
        y_real=y_test,
        y_pred=y_pred,
        title=f"Confusion Matrix\n{name}-{round(THRESHOLD,2)}",
        ax=ax,
    )


In [ ]:
import matplotlib.pyplot as plt
import math

ncols = 4

size = len(predictions)
rows = math.ceil(size / ncols)
width, height = (10, 6)

fig = plt.figure(figsize=(min(ncols * width, 50), rows * height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

for idx, (name, y_pred) in enumerate(predictions.items(), start=1):
    ax = fig.add_subplot(rows, ncols, idx)
    plot_classification_report_sns(
        y_real=y_test,
        y_pred=y_pred,
        title=f"Classification Report\n{name}-{round(THRESHOLD,2)}",
        ax=ax,
    )


In [ ]:
import matplotlib.pyplot as plt
import math

ncols = 4

size = len(predictions_proba)
rows = math.ceil(size / ncols)
width, height = (10, 6)

fig = plt.figure(figsize=(min(ncols * width, 50), rows * height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

for idx, (name, y_pred) in enumerate(predictions_proba.items(), start=1):
    ax = fig.add_subplot(rows, ncols, idx)
    plot_roc_sns(
        y_real=y_test,
        y_pred=y_pred,
        title=f"ROC CURVE\n{name}-{round(THRESHOLD,2)}",
        ax=ax,
    )


In [ ]:
import matplotlib.pyplot as plt
import math

ncols = 4

size = len(predictions_proba)
rows = math.ceil(size / ncols)
width, height = (10, 6)

fig = plt.figure(figsize=(min(ncols * width, 50), rows * height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

for idx, (name, y_pred) in enumerate(predictions_proba.items(), start=1):
    ax = fig.add_subplot(rows, ncols, idx)
    plot_precision_recall_vs_threshold(
        y_real=y_test,
        y_pred=y_pred,
        title=f"Precision and Recall X Threshold\n{name}-{round(THRESHOLD,2)}",
        ax=ax,
    )


In [ ]:
import seaborn as sns

ncols = 4

size = len(predictions_proba)
rows = math.ceil(size / ncols)
width, height = (10, 6)

fig = plt.figure(figsize=(min(ncols * width, 50), rows * height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

for idx, (name, y_pred) in enumerate(predictions_proba.items(), start=1):
    ax = fig.add_subplot(rows, ncols, idx)
    df_proba = pd.concat(
        [pd.Series(y_pred, name="proba", index=y_test.index), y_test], axis=1
    )
    try:
        sns.histplot(data=df_proba, x="proba", hue=y_test.name, ax=ax)
        ax.set_title(f"PROBABILITY DISTRIBUTION \n{name}", fontsize=20)
    except:
        pass


In [ ]:
import matplotlib.pyplot as plt
import math

ncols = 4

size = len(predictions_proba)
rows = math.ceil(size / ncols)
width, height = (10, 6)

fig = plt.figure(figsize=(min(ncols * width, 50), rows * height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

for idx, (name, y_pred) in enumerate(predictions_proba.items(), start=1):
    ax = fig.add_subplot(rows, ncols, idx)
    plot_calibration_curve(
        y_real=y_test,
        y_pred=y_pred,
        title=f"Calibration CURVE\n{name}",
        ax=ax,
        n_bins=20,
    )


In [ ]:
from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
import math

ncols = 4

size = len(predictions_proba)
rows = math.ceil(size / ncols)
width, height = (10, 6)

fig = plt.figure(figsize=(min(ncols * width, 50), rows * height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

for idx, (name, y_pred) in enumerate(predictions_proba.items(), start=1):
    ax = fig.add_subplot(rows, ncols, idx)
    bc = BinaryClassification(
        y_true=y_test, y_pred=y_pred, labels=["Class 0", "Class 1"], threshold=THRESHOLD
    )
    bc.plot_class_distribution()
    title: ax.set_title(f"CLASS DISTRIBUTION\n{name}-{round(THRESHOLD,2)}", fontsize=20)


# Tunning Hyperparameters

In [ ]:
chosen_model_name = "BORUTA_LGBMClassifier"

chosen_model = read_joblib(osp.join(MODELS_DIR, chosen_model_name, "model.p"))
pipe = clone(chosen_model)


In [ ]:
# reading files

df_train = pd.read_parquet(osp.join(BORUTA_PATH, "df_train.parquet"))
df_test = pd.read_parquet(osp.join(BORUTA_PATH, "df_test.parquet"))

logger.info(f"df_train shape: {df_train.shape}")
logger.info(f"df_test shape: {df_test.shape}")

X_train = df_train.drop([TARGET], axis=1)
y_train = df_train[TARGET]
X_test = df_test.drop([TARGET], axis=1)
y_test = df_test[TARGET]


In [ ]:
pipe


## Randomized Search CV

In [ ]:
preprocessor_search_space = {
    "preprocessor__float__imputer": [
        SimpleImputer(strategy="constant", fill_value=0),
        # SimpleImputer(strategy="constant", fill_value=np.nan),
        SimpleImputer(strategy="mean"),
        SimpleImputer(strategy="median"),
        # IterativeImputer(
        #     random_state=0
        #     , estimator=LGBMRegressor(max_depth = 3, random_state=SEED)
        #     , max_iter=10
        #     , tol=1e-1
        #     )
    ],
    "preprocessor__float__scaler": [
        FunctionTransformer(),
        # FunctionTransformer(func = np.log1p, inverse_func = np.expm1),
        PowerTransformer(method="yeo-johnson", standardize=False),
        # PowerTransformer(method='box-cox', standardize=False),
        MinMaxScaler(),
        StandardScaler(),
    ],
    "preprocessor__int__imputer": [
        SimpleImputer(strategy="constant", fill_value=0),
        # SimpleImputer(strategy="constant", fill_value=np.nan),
        SimpleImputer(strategy="most_frequent"),
        SimpleImputer(strategy="mean"),
        SimpleImputer(strategy="median"),
        # IterativeImputer(
        #     random_state=0
        #     , estimator=LGBMRegressor(max_depth = 3, random_state=SEED)
        #     , max_iter=10
        #     , tol=1e-1
        #     )
    ],
    "preprocessor__int__scaler": [
        FunctionTransformer(),
        # FunctionTransformer(func = np.log1p, inverse_func = np.expm1),
        PowerTransformer(method="yeo-johnson", standardize=False),
        # PowerTransformer(method='box-cox', standardize=False),
        MinMaxScaler(),
        StandardScaler(),
    ],
}


In [ ]:
search_space = [
    {
        "model": [LGBMClassifier(random_state=SEED)],
        "model__n_estimators": [30, 100, 400, 700, 1000],
        "model__colsample_bytree": [0.7, 0.8],
        "model__max_depth": [3, 5, 10, 15, 20],
        "model__num_leaves": [50, 100, 200],
        "model__reg_alpha": [1.1, 1.2, 1.3],
        "model__reg_lambda": [1.1, 1.2, 1.3],
        "model__min_split_gain": [0.3, 0.4],
        "model__min_child_samples": [10, 20, 30, 40, 50],
        "model__subsample": [0.7, 0.8, 0.9],
        "model__subsample_freq": [20, 10],
        **preprocessor_search_space,
    }
]


In [ ]:
SAMPLE_SIZE = 1000000

X_train_sample = X_train.sample(min(SAMPLE_SIZE, X_train.shape[0]))
# X_train_sample = X_train
y_train_sample = y_train.loc[X_train_sample.index]

optimizer = None
optimizer = RandomizedSearch(
    pipe=pipe,
    search_space=search_space,
    X_train=X_train_sample,
    y_train=y_train_sample,
    scoring=metrics,
    refit="roc_auc",
    n_iter=100,
    n_folds=N_FOLDS,
    verbose=1,
)

print("Finished fitting optimizer")
best_model = optimizer.best_estimator_

print("Retraining best model")

best_model.fit(X_train, y_train)

write_joblib(best_model, osp.join(MODELS_DIR, "best_model.joblib"))


In [ ]:
best_model


In [ ]:
y_pred_train = best_model.predict(X_train)
y_proba_train = best_model.predict_proba(X_train)
y_pred_test = best_model.predict(X_test)
y_proba_test = best_model.predict_proba(X_test)

max_metric_len = max([len(metric_name) for metric_name in metrics])
msg_metrics = f""
for metric_name, scorer in metrics.items():
    train_scorer = get_metric_result(
        scorer, y_train, y_pred_train, y_proba=y_proba_train
    )
    test_scorer = get_metric_result(scorer, y_test, y_pred_test, y_proba=y_proba_test)

    msg_metrics += f" - {metric_name.upper()}{' '*(max_metric_len-len(metric_name))}\t->\tTRAIN: {train_scorer:.2f} | TEST:{test_scorer:.2f}\n"

print(msg_metrics)


# Best Model

In [ ]:
best_model = read_joblib(osp.join(MODELS_DIR, "best_model.joblib"))

name = chosen_model_name

y_pred_train = best_model.predict(X_train)
y_proba_train = best_model.predict_proba(X_train)[:, 1]
y_pred_test = best_model.predict(X_test)
y_proba_test = best_model.predict_proba(X_test)[:, 1]


In [ ]:
y = y_test
y_pred = y_pred_test
y_proba = y_proba_test


## Metrics

### Precision and Recall x Threshold

In [ ]:
import matplotlib.pyplot as plt
import math

width, height = (10, 6)

fig = plt.figure(figsize=(min(width, 50), height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

ax = fig.add_subplot(1, 1, 1)

plot_precision_recall_vs_threshold(
    y_real=y, y_pred=y_proba, title=f"Precision and Recall X Threshold\n{name}", ax=ax
)


### Calibration Curve

In [ ]:
width, height = (10, 6)

fig = plt.figure(figsize=(min(width, 50), height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

ax = fig.add_subplot(1, 1, 1)

plot_calibration_curve(
    y_real=y, y_pred=y_proba, title=f"Calibration CURVE\n{name}", ax=ax, n_bins=20
)


### Probability Distribution

In [ ]:
width, height = (10, 6)

fig = plt.figure(figsize=(min(width, 50), height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

ax = fig.add_subplot(1, 1, 1)

df_proba = pd.concat([pd.Series(y_proba, name="proba", index=y.index), y], axis=1)

sns.histplot(data=df_proba, x="proba", hue=y.name, ax=ax)
ax.set_title(f"PROBABILITY DISTRIBUTION \n{name}", fontsize=20)

plt.show()


### Confusion Matrix x Threshold

In [ ]:
import matplotlib.pyplot as plt
import math

ncols = 5
THRESHOLDS = np.arange(0.05, 1.05, 0.05)

size = len(THRESHOLDS)
rows = math.ceil(size / ncols)
width, height = (10, 6)

fig = plt.figure(figsize=(min(ncols * width, 50), rows * height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

for idx, threshold in enumerate(THRESHOLDS, start=1):
    ax = fig.add_subplot(rows, ncols, idx)
    y_pred_threshold = best_model.predict_proba(X_test)[:, 1] > threshold
    plot_confusion_matrix(
        y_real=y,
        y_pred=y_pred_threshold,
        title=f"Confusion Matrix\n{name}-{round(threshold,2)}",
        ax=ax,
    )


### Classification Report x Threshold

In [ ]:
import matplotlib.pyplot as plt
import math

ncols = 5
THRESHOLDS = np.arange(0.05, 1.05, 0.05)

size = len(THRESHOLDS)
rows = math.ceil(size / ncols)
width, height = (10, 6)

fig = plt.figure(figsize=(min(ncols * width, 50), rows * height))
fig.subplots_adjust(hspace=0.7, wspace=0.2)

for idx, threshold in enumerate(THRESHOLDS, start=1):
    ax = fig.add_subplot(rows, ncols, idx)
    y_pred_threshold = best_model.predict_proba(X_test)[:, 1] > threshold
    plot_classification_report_sns(
        y_real=y,
        y_pred=y_pred_threshold,
        title=f"Classification Report\n{name}-{round(threshold,2)}",
        ax=ax,
    )


### Defining Threshold

In [ ]:
THRESHOLD = 0.35
y_pred = best_model.predict_proba(X_test)[:, 1] > THRESHOLD


### Confusion Matrix

In [ ]:
plot_confusion_matrix(
    y_real=y, y_pred=y_pred, title=f"Confusion Matrix\n{name}-{round(THRESHOLD,2)}"
)


### Classification Report

In [ ]:
plot_classification_report_sns(
    y_real=y, y_pred=y_pred, title=f"Classification Report\n{name}-{round(THRESHOLD,2)}"
)


### Roc Curve

In [ ]:
from plot_metric.functions import BinaryClassification

f, ax = plt.subplots(figsize=(10, 6))
bc = BinaryClassification(
    y_true=y, y_pred=y_proba, labels=["Class 1", "Class 2"], threshold=THRESHOLD
)
bc.plot_roc_curve()
title: ax.set_title(f"ROC CURVE\n{name}-{round(THRESHOLD,2)}", fontsize=20)


### Precision x Recall

In [ ]:
from plot_metric.functions import BinaryClassification

f, ax = plt.subplots(figsize=(10, 6))
bc = BinaryClassification(
    y_true=y, y_pred=y_proba, labels=["Class 0", "Class 1"], threshold=THRESHOLD
)
bc.plot_precision_recall_curve()
title: ax.set_title(f"PRECISION X RECALL\n{name}-{round(THRESHOLD,2)}", fontsize=20)


### Class Distribution

In [ ]:
from plot_metric.functions import BinaryClassification

f, ax = plt.subplots(figsize=(10, 6))
bc = BinaryClassification(
    y_true=y, y_pred=y_proba, labels=["Class 0", "Class 1"], threshold=THRESHOLD
)
bc.plot_class_distribution()
title: ax.set_title(f"CLASS DISTRIBUTION\n{name}-{round(THRESHOLD,2)}", fontsize=20)


## Shap

In [ ]:
import shap

shap.initjs()

model = best_model

# Using a random sample of the dataframe for better time computation
X_train_sampled = X_train.sample(30, random_state=10)
X_test_sampled = X_test.sample(1000, random_state=10)


# explain the model's predictions using SHAP values
def model_predict(data_asarray, model=model, columns=X_train_sampled.columns):
    data_asframe = pd.DataFrame(data_asarray, columns=columns)
    predictions = model.predict_proba(data_asframe)
    return predictions


explainer = shap.KernelExplainer(
    model=model_predict, data=X_train_sampled, link="identity"
)
shap_values = explainer.shap_values(X_test_sampled)


In [ ]:
shap.summary_plot(shap_values[1], X_test_sampled, plot_size=(18, 10))


In [ ]:
shap.force_plot(
    explainer.expected_value[0],
    shap_values[0][0],
    feature_names=X_train_sampled.columns,
    link="logit",
)


In [ ]:
shap.plots._waterfall.waterfall_legacy(
    explainer.expected_value[1],
    shap_values[1][0],
    feature_names=X_train_sampled.columns,
)


In [ ]:
shap.summary_plot(
    shap_values,
    plot_type="bar",
    feature_names=X_train_sampled.columns,
    plot_size=(18, 10),
)


In [ ]:
shap.decision_plot(
    explainer.expected_value[1],
    shap_values[1][:500],
    feature_names=X_test_sampled.columns.to_list(),
    xlim=(-0.1, 1),
    feature_order="hclust",
)


In [ ]:
shap.plots.force(
    explainer.expected_value[1],
    shap_values[1],
    feature_names=X_test_sampled.columns.to_list(),
)
